In [1]:
import requests, json, os, time
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [2]:
wiki = 'https://en.wikipedia.org'

def get_category(url, is_next=False): 
    data = {}
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # Subcategories
    if soup.select('#mw-subcategories .mw-content-ltr a'): # subcategory
        for word in soup.select('#mw-subcategories .mw-content-ltr a'): 
            print(word.text)
            print(wiki + word['href'])
            get_category(wiki + word['href'])
    
    if soup.select('#mw-subcategories a'): # subcategory have next page ?
        if soup.select('#mw-subcategories a')[0].text == 'next page':
            print(wiki + soup.select('#mw-subcategories a')[0]['href'])
            get_category(wiki + soup.select('#mw-subcategories a')[0]['href'])
    
    # Pages
    if soup.select('#mw-pages .mw-content-ltr a'): # page in category
        for word in soup.select('#mw-pages .mw-content-ltr a'): 
            print(word.text)
            print(wiki + word['href'])
            data[word.text] = get_page(wiki + word['href'])
    
    if soup.select('#mw-pages a'): # page have next page ?
        if soup.select('#mw-pages a')[1].text == 'next page':
            print(wiki + soup.select('#mw-pages a')[1]['href'])
            get_category(wiki + soup.select('#mw-pages a')[1]['href'], True)
    
    # output json   
    file_name = soup.select('#firstHeading')[0].text.split('Category:')[1]
    if(is_next == True): # have next page
        with open( dir_name + '/' + file_name + '.json', 'r') as f:
            origin_data = json.load(f)
            origin_data.update(data)
            data = origin_data
        with open( dir_name + '/' + file_name + '.json', 'w') as f:  
            json.dump(data, f)
    else:
        with open( dir_name + '/' + file_name + '.json', 'w') as f:
            json.dump(data , f)

In [3]:
def get_page(url):
    page_res = requests.get(url)
    page_soup = BeautifulSoup(page_res.text, 'html.parser')
    contents = ''
    
    for word in page_soup.select('p'):
        # print(word.text)
        contents += word.text
        
    print()
    return contents

In [4]:
def crawler(category):
    
    # create dir
    global dir_name 
    dir_name = "jsondata-" + time.strftime('%Y-%m-%d %H%M%S') 
    if not os.path.exists(dir_name):    
        os.makedirs(dir_name)
    
    url = 'https://en.wikipedia.org/wiki/Category:' + category
    print('Start get ' + category)
    get_category(url)

In [5]:
# wiki category 
crawler('Porsche_vehicles')

Start get Porsche_vehicles
Porsche concept vehicles
https://en.wikipedia.org/wiki/Category:Porsche_concept_vehicles
Porsche 114
https://en.wikipedia.org/wiki/Porsche_114

Porsche 989
https://en.wikipedia.org/wiki/Porsche_989

Porsche C88
https://en.wikipedia.org/wiki/Porsche_C88

Porsche Mission E
https://en.wikipedia.org/wiki/Porsche_Mission_E

Porsche Panamericana
https://en.wikipedia.org/wiki/Porsche_Panamericana

Porsche Typ 754 T7
https://en.wikipedia.org/wiki/Porsche_Typ_754_T7

Porsche Tapiro
https://en.wikipedia.org/wiki/Porsche_Tapiro

Porsche racing cars
https://en.wikipedia.org/wiki/Category:Porsche_racing_cars
Porsche Formula One cars
https://en.wikipedia.org/wiki/Category:Porsche_Formula_One_cars
Porsche 718
https://en.wikipedia.org/wiki/Porsche_718

Porsche 787
https://en.wikipedia.org/wiki/Porsche_787

Porsche 804
https://en.wikipedia.org/wiki/Porsche_804

Porsche 550
https://en.wikipedia.org/wiki/Porsche_550

Porsche 718
https://en.wikipedia.org/wiki/Porsche_718

Porsch

In [6]:
def keyword_collection():
    # read all data in dir
    files= os.listdir(dir_name)
    for file in files:
        print(file)
        with open (dir_name + '/' + file, 'r') as f:
            data = json.load(f)
            keyword_filter(data)

In [7]:
stop_symbols = ['–', '●', '○', '～', '『', '』', '】', '【', '「', '」', 'ˊ', '’', '‘', '﹕', '﹐', '.', '…', '\t', '\n', ' ', ']', '[', ',', '?', '、', '。', '“', '”', '《', '》', '！', '，', '：', '；', '？', '\'', ';', '\"', '{', '}', '/', '(', ')', '\"\"', '``', '\'\'', '\'m', '\'s', ':', '&', '@', '#', '%']
upper_stop_words = []
for word in stopwords.words('english'):
    upper_stop_words.append( word[0].upper() + word[1:len(word)])
stop_words = stopwords.words('english') + upper_stop_words +stop_symbols
print(stop_words)

def keyword_filter(datas):
    datas = list(datas.values()) # get values in dict
    category_keywords = {}
    
    for page_data in datas:
        
        #sentence tokenize
        sentences = sent_tokenize(page_data)
        #print(sentences)
        
        #word tokenize
        for sent in sentences:
            word_list = word_tokenize(sent)
            word_list = list( set(word_list) - set(stop_words) ) # delete stop word
            word_list.sort()
            print(word_list)
            keywords = gen_keywords(word_list)
            
            # store keywords in category_keywords
            for k in keywords:
                if k in category_keywords:
                    for k2 in keywords[k]:
                        if k2 in category_keywords[k]: # same keyword update value
                            category_keywords[k][k2] += keywords[k][k2]
                        else:
                            category_keywords[k][k2] = keywords[k][k2]
                else:
                    category_keywords[k] = keywords[k]
        
    
    # create keyword dir 
    dir_name = "keywords" 
    
    if not os.path.exists(dir_name): # first time generate keyword data
        os.makedirs(dir_name)
        with open( dir_name + '/keywords.json', 'w') as f:  
            json.dump(keywords_sort(category_keywords), f)
            
    else: # keyword data had existed
        origin_keywords = {}
        with open( dir_name + '/keywords.json', 'r') as f: # update keyword data
            origin_keywords = json.load(f) 
            
            for k in category_keywords:
                if k in origin_keywords:
                    for k2 in category_keywords[k]:
                        if k2 in origin_keywords[k]: # same keyword update value
                            origin_keywords[k][k2] += category_keywords[k][k2]
                        else:
                            origin_keywords[k][k2] = category_keywords[k][k2]
                else:
                    origin_keywords[k] = category_keywords[k]
                            
                               
        with open( dir_name + '/keywords.json', 'w') as f:  
            json.dump(keywords_sort(origin_keywords), f)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
def gen_keywords(word_list):
    keywords = {}
    
    # init keyword data
    for i in word_list:  
        count = {}
        for j in word_list:
            if i != j:
                count[j] = 1
        keywords[i] = count # {'i': {'j1': time, 'j2': time ...}}
    
    return keywords

In [9]:
def keywords_sort(keywords):
    sorted_keywords = {}
    for k in keywords:
        sorts = sorted(keywords[k].items(), key=lambda x:x[1], reverse=True)
        sorted_keywords[k] = {sorts[i][0] : sorts[i][1] for i in range(0, len(sorts))}
    return sorted_keywords

In [10]:
keyword_collection()

Porsche Formula One cars.json
['1957', '1962', '718', 'Porsche', 'built', 'car', 'racing']
['550A', '718', 'Porsche', 'body', 'development', 'improvements', 'made', 'successful', 'suspension', 'work']
["'Kurz", 'German', 'RSK', 'RennSport', 'build', 'car', 'era', 'led', 'predecessors', 'referred', 'shorter', 'typical', 'wheelbase']
['1.5-litre', '106', '142', '547/3', '550A', 'Type', 'engine', 'horsepower', 'introduced', 'kW', 'layout', 'mid-engined', 'quad-cam', 'used']
['1957', '24', 'Barth', 'Edgar', 'Hours', 'Le', 'Maglioli', 'Mans', 'Umberto', 'car', 'debut', 'driven', 'made', 'racing']
['accident', 'car', 'due', 'failed', 'finish', 'race']
['1958', 'Behra', 'Hans', 'Herrmann', 'Jean', 'Le', 'Mans', 'car', 'class', 'finished', 'first', 'guided', 'overall', 'third']
['Behra', 'Florio', 'Jean', 'Targa', 'also', 'brought', 'cars', 'home', 'one', 'second']
['1959', 'Barth', 'Edgar', 'Florio', 'Seidel', 'Targa', 'Wolfgang', 'car', 'claimed', 'driven', 'overall', 'victory']
['1958', '19

['108', '110', '121', '2', '3', '4', '5', '5000', '6200', '81', '89', 'N⋅m', 'PS', 'first', 'hp', 'kW', 'lb⋅ft', 'maximum', 'produced', 'rpm', 'torque', 'version']
['550', 'axle', 'engine', 'front', 'making', 'mid-engined', 'mounted', 'rear']
['allows', 'balanced', 'distribution', 'gives', 'handling', 'largely', 'neutral', 'weight']
['axis', 'car', 'control', 'difficult', 'hand', 'inertia', 'lead', 'low', 'mass', 'moment', 'rotation', 'sudden', 'vehicle', 'vertical']
['1930s', 'Auto', 'Grand', 'Porsche', 'Prix', 'Union', 'already', 'car', 'design', 'late', 'principle', 'used']
['4-speed', '550', 'first', 'fully', 'gearbox', 'synchronized']
['1956', 'Starting', 'barrier', 'first', 'five-speed', 'gear', 'gearbox', 'like', 'placed', 'reverse', 'start', 'synchronized', 'used']
['Excessive', 'corners', 'differential', 'drive', 'limited', 'prevented', 'slip', 'wheels']
['1951', '356', '6', 'Ferry', 'Glöckler', 'Inspired', 'Porsche', 'Walter', 'auto', 'build', 'built', 'car', 'created', 'deci

['0', 'French', 'Peugeot', 'Porsche', 'already', 'asserting', 'car', 'digit', 'key', 'maker', 'many', 'markets', 'middle', 'models', 'naming', 'number', 'objected', 'ownership', 'rights', 'scheme', 'sold', 'three', 'using']
['0', '1', '911', 'Porsche', 'called', 'car', 'middle', 'replaced', 'simply']
['901s', 'Officially', 'Porsche', 'already', 'constructed', 'customers', 'exhibitions', 'none', 'private', 'sold', 'testing', 'used']
['1', '1988', '20', 'Kurt', 'Lori', 'Nevertheless', 'Porsche', 'Schneider', 'appear', 'cars', 'completely', 'discovered', 'made', 'number', 'ownership', 'private', 'restored', 'retained', 'several', 'subsequently', 'time', 'wife']
['2', '2010', '37', 'Alois', 'Porsche', 'Ruf', 'car', 'named', 'number', 'owned', 'reported', 'specialist']
['1', 'Also', 'Porsche', 'affected', 'also', 'cars', 'intended', 'legal', 'models', 'primarily', 'racing', 'road', 'sold']
['90x', 'Porsche', 'car', 'cars', 'continue', 'design', 'digit', 'enthusiasts', 'internal', 'kept', 'l

['2012', '911', '991', '997', 'Porsche', 'depending', 'either', 'month', 'production']
['3.6', '353', '457', '473', '480', '620', '996', '997', 'L', 'N⋅m', 'PS', 'Turbo', 'Turbocharged', 'bhp', 'engine', 'featured', 'kW', 'lb⋅ft', 'modifications', 'produce', 'series', 'torque', 'twin-turbocharged', 'version']
['VTG', 'boost', 'combines', 'geometry', 'high-rev', 'larger', 'low-rev', 'power', 'quick', 'responses', 'small', 'turbine', 'turbocharger', 'variable']
['996', 'Turbo', 'also', 'compared', 'efficiency', 'fuel', 'higher', 'offers']
['997', 'Cayenne', 'Porsche', 'Turbo', 'all-wheel-drive', 'features', 'found', 'new', 'one', 'similar', 'system']
['Management', 'PTM', 'Porsche', 'Traction', 'amount', 'avoid', 'clutch-based', 'incorporates', 'new', 'slippage', 'system', 'tire', 'torque', 'varies', 'wheels']
['According', 'Porsche', 'conditions', 'control', 'greatly', 'handling', 'improved', 'neutral', 'oversteer', 'performance', 'redirecting', 'results', 'torque', 'understeer', 'weath

['991', 'GT3', 'adapted', 'components', 'crankshaft', 'developed', 'particularly', 'specially', 'train', 'valve']
['55', '991', 'GT3', 'another', 'based', 'cars', 'developed', 'dual-clutch', 'feature', 'gearbox', 'manual', 'racing', 'sequential', 'specially', 'transmissions', 'used']
['0-60', '11.2', '126', '203', '3.1', '56', '911', '97', 'GT3', 'able', 'accelerate', 'claimed', 'km/h', 'less', 'mile', 'mph', 'quarter', 'seconds']
['202', '325', '57', '58', 'GT3', 'claimed', 'km/h', 'mph', 'speed', 'top']
['2017', '59', '991.2', 'GT3', 'Geneva', 'Motor', 'Porsche', 'Show', 'facelifted', 'unveiled']
['4.0', '9000', 'Extensive', 'allowing', 'changes', 'engine', 'flat-six', 'litre', 'made', 'redline', 'rpm']
['Porsche', 'focus', 'friction', 'improve', 'internal', 'reducing', 'response', 'throttle']
['991.1', 'Compared', 'back', 'effective', 'higher', 'located', 'rear', 'spoiler']
['changes', 'front', 'new', 'rear', 'spoiler', 'suspension']
['991.2', 'GT3', 'PDK', 'back', 'brought', 'choic

...

Porsche concept vehicles.json
['114', '1493', '1938', 'Porsche', 'V10', 'car', 'cc', 'design', 'engine', 'powered', 'proposed', 'sports']
['Ferdinand', 'German', 'KDF', 'Porsche', 'VW', 'Volkswagen', 'building', 'car', 'considered', 'designing', 'government', 'program', 'sports', 'version']
['1.0L', '1.5L', '100', '160', '64', 'Internally', 'Porsche', 'Type', 'Volkswagen', 'aluminium', 'body', 'car', 'engine', 'feature', 'km/h', 'known', 'least', 'mph', 'original', 'speed', 'streamlined', 'top', 'version', 'would']
['German', 'Originally', 'VW', 'car', 'company', 'due', 'government', 'intricacies', 'large', 'law', 'legal', 'made', 'number', 'parts', 'private', 'sell', 'use']
['1938', '64', 'Ferdinand', 'Type', 'VW', 'arrange', 'given', 'parts', 'supply', 'trying']
['Ferdinand', 'Ferry', 'Porsche', 'built', 'car', 'decided', 'ground', 'internally', 'redesign', 'son']
['Porsche', 'actually', 'built', 'car', 'first', 'would']
['114', '64', 'F-Wagen', 'Ferry', 'Known', 'P-Wagen', 'Type', '

Porsche racing cars.json
['1953-1956', '550', 'Porsche', 'car', 'produced', 'racing', 'sports']
['1,498', '1.5', '4', '547', '550/550', '91.4', 'Engine', 'Fuhrmann', 'L', 'Type', 'aircooled', 'aluminum', 'aspirated', 'boxer', 'cc', 'cu', 'cylinder', 'engine', 'known', 'naturally', 'powered']
['1', '2', '2-barrel', 'DOHC', 'PJJ', 'Solex', 'camshafts', 'carburetors', 'coils', 'configuration', 'cylinder', 'double', 'driven', 'dual', 'equipped', 'fall', 'four', 'gasifiers', 'ignition', 'manifolds', 'per', 'separate', 'shafts', 'side', 'twin', 'two', 'uses', 'valves', 'valvetrain', 'vertical', 'well']
['108', '110', '121', '2', '3', '4', '5', '5000', '6200', '81', '89', 'N⋅m', 'PS', 'first', 'hp', 'kW', 'lb⋅ft', 'maximum', 'produced', 'rpm', 'torque', 'version']
['550', 'axle', 'engine', 'front', 'making', 'mid-engined', 'mounted', 'rear']
['allows', 'balanced', 'distribution', 'gives', 'handling', 'largely', 'neutral', 'weight']
['axis', 'car', 'control', 'difficult', 'hand', 'inertia', 'l

['2017', '991', 'GT3', 'Porsche', 'R', 'front']
['6-cylinder', '914', '914-6', 'GT', 'Porsche', 'based', 'built', 'car', 'engine', 'model', 'package', 'race']
['914-6', 'GT', 'configured', 'race', 'version']
['GT', 'box-like', 'distinguishable', 'factory', 'fender', 'flares', 'offered', 'option', 'steel']
['914-6', 'GT', 'became', 'configurations', 'different', 'employing', 'engine', 'known', 'quickly', 'raced']
["'T", '911', 'basic', 'engine', 'included', 'specification']
['6', 'Another', 'Carrera', 'configuration', 'converted', 'engine', 'popular', 'use']
['1970', '24', '914-6', 'GT', 'Hours', 'Le', 'Mans', 'Porsche', 'entered', 'factory', 'grueling', 'option', 'race']
['40', 'Ballot-Léna', 'Chasseuil', 'Claude', 'Guy', 'car', 'driven', 'identified']
['512S', '6th', '917K', 'Ferrari', 'Flat-12', 'Porsche', 'V12', 'finished', 'like', 'overall', 'preceded', 'prototype', 'racecars']
['40', '911S', 'Porsche', 'addition', 'ahead', 'class', 'entries', 'several']
['914-6s', 'also', 'compete

['1978', '935', '935/78', 'Le', 'Mans', 'developed', 'final', 'intended', 'third', 'version']
['1977', '1978', 'Due', 'Porsche', 'air', 'biturbo', 'cooling', 'cylinder', 'engine', 'equipped', 'failures', 'four', 'gasket', 'head', 'heads', 'introduced', 'parted', 'tradition', 'valves', 'water-cooled', 'well']
['3.2', '750', '845', 'L', 'capacity', 'enlarged', 'hp', 'increasing', 'output', 'typically']
['1030', '2', 'Le', 'Mans', 'important', 'kg', 'less', 'straights', 'weight']
['1978', 'Dunlop', 'Mulsanne', 'Rouge', 'Tertre', 'another', 'better', 'clockwise', 'corners', 'distinctive', 'distribution', 'driver', 'feature', 'like', 'model', 'moved', 'right', 'right-hand', 'run', 'seat', 'side', 'sight', 'track', 'weight']
['5', 'BMW', 'Group', 'accommodate', 'advantage', 'allowing', 'car', 'cut', 'engine', 'exhaust', 'floor', 'front', 'full', 'introduced', 'loophole', 'new', 'rules', 'took']
['10', '3.9', '911', 'Porsche', 'also', 'applied', 'away', 'body', 'cm', 'cut', 'engine', 'floor',

['1995', 'IMSA', 'Porsche', 'Unfortunately', 'WSC', 'canceling', 'changed', 'leading', 'prior', 'project', 'regulations', 'season']
['1996', '24', 'February', 'Hours', 'However', 'Joest', 'Le', 'Mans', 'Porsche', 'Racing', 'Reinhold', 'compete', 'convinced', 'give', 'prototype', 'team', 'unused']
['Joest', 'LMP1', 'Le', 'Mans', 'Porsche', 'Prototype', 'allow', 'approval', 'car', 'construction', 'existing', 'forth', 'meet', 'modifications', 'money', 'order', 'put', 'regulations', 'scratch', 'second', 'slight', 'well']
['Joest', 'Porsche', 'agreed', 'car', 'development', 'help', 'pay', 'services']
['1996', '1997', '24', 'Following', 'Hours', 'Le', 'Mans', 'Porsche', 'WSC-95s', 'decided', 'project', 'success', 'take', 'winning']
['WSC-95', 'bodywork', 'chassis', 'major', 'revisions', 'underwent']
['air', 'allowed', 'engine', 'intakes', 'large', 'middle', 'moving', 'necessitating', 'nose', 'raised', 'removal', 'rollbar', 'scoop', 'sculpting', 'sides', 'underneath']
['also', 'car', 'covered

In [11]:
def get_keyword(keyword):
    with open( 'keywords/keywords.json', 'r') as f:
        keywords = json.load(f)
        if keyword in keywords:
            return keywords[keyword]
        else:
            return 'Not find'

In [12]:
get_keyword('Porsche')

{'car': 308,
 '911': 254,
 'engine': 199,
 'GT3': 186,
 'new': 167,
 'first': 141,
 'racing': 136,
 'model': 130,
 'also': 117,
 'cars': 117,
 'RS': 110,
 'version': 106,
 'Carrera': 102,
 'front': 101,
 'models': 95,
 '997': 94,
 'race': 91,
 'Boxster': 90,
 'rear': 90,
 'Le': 90,
 'Mans': 90,
 'Turbo': 86,
 'used': 81,
 'production': 80,
 'introduced': 76,
 'built': 74,
 'would': 73,
 'sports': 72,
 'year': 67,
 'kW': 67,
 '996': 65,
 'made': 63,
 'Cayman': 61,
 'time': 60,
 'designed': 58,
 'produced': 56,
 'hp': 55,
 'Hours': 55,
 '356': 54,
 'Spyder': 54,
 'developed': 54,
 'factory': 54,
 '4': 52,
 'design': 52,
 '2': 52,
 '24': 52,
 'generation': 51,
 '1': 51,
 'Show': 50,
 'system': 50,
 'Group': 50,
 'transmission': 47,
 'PS': 47,
 'one': 47,
 'two': 47,
 'chassis': 47,
 'standard': 46,
 'vehicle': 44,
 'available': 43,
 'based': 43,
 '5': 43,
 '928': 42,
 'body': 42,
 'use': 42,
 'entered': 42,
 'Championship': 42,
 '718': 41,
 '917': 41,
 'development': 40,
 'suspension': 40